In [163]:
import pandas as pd
import json
import os
import warnings
warnings.filterwarnings('ignore')


In [170]:
# Initializing global variables
SOURCES = ['glassdoor', 'indeed']
COMPANIES = ['airbnb', 'booking', 'tripadvisor']
SOURCE_N_FILES = {'glassdoor':20, 'indeed':10}

## Aggregating all JSON files to CSV files for each company from every source

In [171]:
def json_to_csv(source, company, n_files):
    path_prefix = f'data/{source}/{company}/{source}_{company}'
    for i in range(n_files):
        json_path = f'{path_prefix}_{i}.json'
        with open(json_path, 'r') as file:
            data = json.load(file)
        data = pd.DataFrame(data['reviews'])
        if not os.path.exists(f'{path_prefix}.csv'):
            data.to_csv(f'{path_prefix}.csv', mode='w', header=True, index=False)
            print(f'{path_prefix}.csv - Created Successfully!')
        else: 
            data.to_csv(f'{path_prefix}.csv', mode='a', header=False, index=False)

In [172]:
# Create Glassdoor data
for company in COMPANIES: 
    json_to_csv('glassdoor', company, SOURCE_N_FILES['glassdoor'])

data/glassdoor/airbnb/glassdoor_airbnb.csv - Created Successfully!
data/glassdoor/booking/glassdoor_booking.csv - Created Successfully!
data/glassdoor/tripadvisor/glassdoor_tripadvisor.csv - Created Successfully!


In [173]:
# Create Indeed data
for company in COMPANIES: 
    json_to_csv('indeed', company, SOURCE_N_FILES['indeed'])

data/indeed/airbnb/indeed_airbnb.csv - Created Successfully!
data/indeed/booking/indeed_booking.csv - Created Successfully!
data/indeed/tripadvisor/indeed_tripadvisor.csv - Created Successfully!


## Calculating averages of categorical themes from employee ratings

In [174]:
def calculate_averages(source, company, columns):
    path = f'data/{source}/{company}/{source}_{company}.csv'
    reviews = pd.read_csv(path)
    means = reviews[columns].mean().round(2)
    means = pd.concat([pd.Series([source,company], index=['source','company']), means])
    return means

### Glassdoor reviews

In [175]:
glassdoor_category_columns = [
        'culture_and_values_rating', 'diversity_and_inclusion_rating', 
        'work_life_balance_rating', 'senior_management_rating', 
        'compensation_and_benefits_rating', 'career_opportunities_rating'
        ]
glassdoor_averages = pd.DataFrame(columns = ['source', 'company']+glassdoor_category_columns)
for company in COMPANIES: 
    means = calculate_averages('glassdoor', company, glassdoor_category_columns)
    glassdoor_averages = pd.concat([glassdoor_averages, pd.DataFrame([means])], ignore_index=True)

In [176]:
glassdoor_averages.to_csv(f'data/glassdoor/glassdoor_averages.csv')
glassdoor_averages

,source,company,culture_and_values_rating,diversity_and_inclusion_rating,work_life_balance_rating,senior_management_rating,compensation_and_benefits_rating,career_opportunities_rating
0,glassdoor,airbnb,3.09,3.12,2.91,2.64,3.08,2.71
1,glassdoor,booking,3.05,3.20,2.92,2.64,2.90,2.59
2,glassdoor,tripadvisor,2.88,3.00,2.88,2.37,2.93,2.48


### Indeed reviews

In [177]:
indeed_category_columns = [
    'job_work_and_life_balance_rating', 'compensation_and_benefits_rating', 
    'job_security_and_advancement_rating', 'management_rating', 'job_culture_rating'
]
indeed_averages = pd.DataFrame(columns = ['source', 'company']+indeed_category_columns)
for company in COMPANIES: 
    means = calculate_averages('indeed', company, indeed_category_columns)
    indeed_averages = pd.concat([indeed_averages, pd.DataFrame([means])], ignore_index=True)

In [178]:
indeed_averages.to_csv(f'data/indeed/indeed_averages.csv')
indeed_averages

,source,company,job_work_and_life_balance_rating,compensation_and_benefits_rating,job_security_and_advancement_rating,management_rating,job_culture_rating
0,indeed,airbnb,1.85,1.78,1.71,1.72,1.89
1,indeed,booking,1.83,1.73,1.60,1.64,1.80
2,indeed,tripadvisor,3.37,3.35,2.75,2.88,3.33


### Combined Averages - Source wise

In [179]:
indeed_averages = indeed_averages.rename(columns={
    'job_work_and_life_balance_rating': 'work_life_balance_rating',
    'compensation_and_benefits_rating': 'compensation_and_benefits_rating',
    'job_security_and_advancement_rating': 'career_opportunities_rating',
    'management_rating': 'senior_management_rating',
    'job_culture_rating': 'culture_and_values_rating'
})

missing_columns = set(glassdoor_averages.columns) - set(indeed_averages.columns)
for col in missing_columns:
    indeed_averages[col] = 0
indeed_averages = indeed_averages[glassdoor_averages.columns]

combined_averages = pd.concat([glassdoor_averages, indeed_averages], ignore_index=True)

In [180]:
combined_averages.to_csv(f'data/aggregated/sourcewise_averages.csv')
combined_averages

,source,company,culture_and_values_rating,diversity_and_inclusion_rating,work_life_balance_rating,senior_management_rating,compensation_and_benefits_rating,career_opportunities_rating
0,glassdoor,airbnb,3.09,3.12,2.91,2.64,3.08,2.71
1,glassdoor,booking,3.05,3.20,2.92,2.64,2.90,2.59
2,glassdoor,tripadvisor,2.88,3.00,2.88,2.37,2.93,2.48
3,indeed,airbnb,1.89,0.00,1.85,1.72,1.78,1.71
4,indeed,booking,1.80,0.00,1.83,1.64,1.73,1.60
5,indeed,tripadvisor,3.33,0.00,3.37,2.88,3.35,2.75


### Combined averages - Overall

In [181]:
def aggregate_ratings(company):
    glassdoor_path = f'data/glassdoor/{company}/glassdoor_{company}.csv'
    glassdoor_columns = [
        'id', 'rating', 'culture_and_values_rating', 'work_life_balance_rating', 
        'senior_management_rating', 'compensation_and_benefits_rating', 'career_opportunities_rating'
        ]
    glassdoor_categorical_ratings = pd.read_csv(glassdoor_path)[glassdoor_columns]
    glassdoor_categorical_ratings.insert(0, 'source', 'glassdoor')

    indeed_path = f'data/indeed/{company}/indeed_{company}.csv'
    indeed_columns = [
        'id', 'rating', 'job_culture_rating', 'job_work_and_life_balance_rating', 
        'management_rating', 'compensation_and_benefits_rating', 'job_security_and_advancement_rating'
        ]
    indeed_categorical_ratings = pd.read_csv(indeed_path)[indeed_columns]
    indeed_categorical_ratings.insert(0, 'source', 'indeed')

    indeed_categorical_ratings = indeed_categorical_ratings.rename(columns={
        'job_work_and_life_balance_rating': 'work_life_balance_rating',
        'compensation_and_benefits_rating': 'compensation_and_benefits_rating',
        'job_security_and_advancement_rating': 'career_opportunities_rating',
        'management_rating': 'senior_management_rating',
        'job_culture_rating': 'culture_and_values_rating'
    })
    indeed_categorical_ratings = indeed_categorical_ratings[glassdoor_categorical_ratings.columns]

    aggregated_categorical_ratings = pd.concat([glassdoor_categorical_ratings, indeed_categorical_ratings]).reset_index(drop=True)
    return aggregated_categorical_ratings

In [182]:
for company in COMPANIES: 
    aggregated_company_ratings = aggregate_ratings(company)
    aggregated_company_ratings.to_csv(f'data/aggregated/aggregated_{company}_ratings.csv')

In [186]:
aggregated_means = []
for company in COMPANIES:
    path = f'data/aggregated/aggregated_{company}_ratings.csv'
    ratings = pd.read_csv(path)
    company_means = ratings[ratings.columns[3:]].mean().round(2)
    company_means = pd.concat([pd.Series([company,len(ratings)], index=['company','n_ratings']), company_means])
    aggregated_means.append(company_means)


## Exporting the aggregated data set

In [187]:
aggregated_means = pd.DataFrame(aggregated_means)
aggregated_means.to_csv(f'data/aggregated/aggregated_mean_ratings.csv')
aggregated_means.head()

,company,n_ratings,rating,culture_and_values_rating,work_life_balance_rating,senior_management_rating,compensation_and_benefits_rating,career_opportunities_rating
0,airbnb,401,4.19,2.49,2.38,2.18,2.43,2.21
1,booking,401,3.78,2.42,2.37,2.14,2.32,2.09
2,tripadvisor,332,3.85,3.06,3.07,2.57,3.10,2.58
